In [1]:
from transformers import (
    set_seed,
)

In [2]:
set_seed(123)

In [3]:
from datasets import load_dataset

data_files = "./train_jp.json"
datasets = load_dataset("json", data_files=data_files, split={
    "train": "train[:80%]",
    "validation": "train[80%:90%]",
    "test": "train[90%:]"
})

print(datasets)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6707
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 745
    })
})


In [4]:
column_names = datasets["train"].column_names
features = datasets["train"].features

column_names, features

(['tokens', 'tags'],
 {'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)})

In [5]:
text_column_name = "tokens"
label_column_name = "tags"

text_column_name, label_column_name

('tokens', 'tags')

In [6]:
# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
    # unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

label_list = get_label_list(datasets["train"][label_column_name])
label_to_id = {l: i for i, l in enumerate(label_list)}
num_labels = len(label_list)

label_list, label_to_id

(['B', 'I', 'O'], {'B': 0, 'I': 1, 'O': 2})

In [7]:
from transformers import AutoConfig, BertTokenizerFast, BertForTokenClassification

model_name_or_path = "./out_new3"

# config = AutoConfig.from_pretrained(
#     model_name_or_path,
#     num_labels=num_labels,
#     finetuning_task="pos",
# )
tokenizer = BertTokenizerFast.from_pretrained(
    model_name_or_path,
)
model = BertForTokenClassification.from_pretrained(
    model_name_or_path,
)

In [8]:
padding = "max_length" # False

In [9]:
# Whether to put the label for one word on all tokens of generated by that word or just on the ""one (in which case the other tokens will have a padding index).
label_all_tokens = False

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_to_id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


# train_dataset = datasets["train"]
# train_dataset = train_dataset.map(
#     tokenize_and_align_labels,
#     batched=True,
#     num_proc=None,
#     load_from_cache_file=None,
# )

eval_dataset = datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=None,
    load_from_cache_file=None,
)

In [12]:
len(eval_dataset)

745

In [15]:
# Metrics
import numpy as np
from datasets import load_metric
from accelerate import Accelerator

# Accelerator(log_with=args.report_to, project_dir=args.output_dir) if args.with_tracking else Accelerator()
accelerator = Accelerator()

device = accelerator.device
metric = load_metric("./my_seqeval.py")


def compute_metrics2(return_entity_level_metrics: bool = False):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }


C:\Users\karam\AppData\Local\Temp\ipykernel_13556\2864575286.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("./my_seqeval.py")


In [16]:
def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

In [17]:
import torch



eval_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=128)
processed_count = 0
with torch.no_grad():    
    for batch in dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        predictions = outputs.logits.argmax(dim=-1)
        predictions_gathered, labels_gathered = accelerator.gather((predictions, labels))

        preds, refs = get_labels(predictions_gathered, labels_gathered)
        # compute_metrics((outputs.logits, labels), return_entity_level_metrics=False)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )  # predictions and preferences are expected to be a nested list of labels, not label_ids
        processed_count += len(input_ids)
        print(processed_count, end="\r")

In [18]:
compute_metrics2()

{'precision': 0.4701195219123506,
 'recall': 0.6519337016574586,
 'f1': 0.5462962962962964,
 'accuracy': 0.9085395205545393}

In [ ]:
("DONE",)

In [1]:
extracted_spans = [['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Akupressur', 'ABAP', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Akupressur', 'Buchhaltung', 'Buchhaltungstechniken'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Sätze anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Anpassung an Veränderungen in der Forstwirtschaft', 'Buchhaltung'], ['Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'ABAP', 'Buchhaltung', 'ABAP', 'Buchhaltung', 'ABAP', 'Akupressur', 'Buchhaltungstechniken', 'Akupressur', 'ABAP', 'ABAP', 'Buchhaltungstechniken', 'Buchhaltung', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Akutversorgung', 'ABAP', 'ein Skript anpassen', 'Akupressur', 'Buchhaltung', 'Computerkomponenten hinzufügen', '3D-Beleuchtung', 'ABAP', 'Buchhaltung', '3D-Beleuchtung', 'Akupressur', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Schauspieltechniken', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing', 'Akutversorgung', 'ABAP', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'ABAP', 'Systemkomponente erwerben', 'Sätze anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Requisiten anpassen', 'Farbe hinzufügen', 'Akupressur', 'Sätze anpassen', 'ein Skript anpassen'], ['ein Skript anpassen', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', '3D-Druckverfahren', 'ABAP', '3D-Beleuchtung', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken', 'Akupressur', 'Akupressur', 'Farbe hinzufügen', '3D-Beleuchtung', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Versicherungsmathematik', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Anpassung an Veränderungen im Marketing', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Buchhaltungstechniken', 'Schauspieltechniken', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltungstechniken'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Sätze anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', '3D-Beleuchtung', '3D-Beleuchtung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Akupressur', 'ein Skript anpassen'], ['Akupressur', 'Akupressur', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Farbe hinzufügen', 'ein Skript anpassen', 'Anpassung an Veränderungen im Marketing', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Buchhaltung', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Akupressur'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', '3D-Beleuchtung', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Buchhaltung', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Präzisionswaffen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ein Skript anpassen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'ABAP', 'Schauspieltechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ABAP', 'Schauspieltechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', '3D-Texturierung', 'Akupressur', 'Antiquitäten erwerben', 'Sätze anpassen', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Text kulturell anpassen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'Akupressur', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Veränderungen im Marketing', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'ABAP', 'Buchhaltung', 'Buchhaltungstechniken', 'Systemkomponente erwerben', '3D-Beleuchtung', 'Buchhaltung', 'Computerkomponenten hinzufügen', 'Systemkomponente erwerben', '3D-Beleuchtung', 'Buchhaltung', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung', '3D-Beleuchtung', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Akupressur', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Sätze anpassen', 'ein Skript anpassen', 'Sätze anpassen', 'Buchhaltungstechniken', 'ABAP', 'Buchhaltung', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', '3D-Beleuchtung', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'Akupressur', 'ABAP', 'ABAP', 'Anpassung an Veränderungen im Marketing', 'ABAP', 'Systemkomponente erwerben', 'ABAP', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', '3D-Modellierung', 'ABAP', 'ABAP', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Buchhaltungstechniken', 'ABAP', 'ABAP', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Akupressur', 'ABAP', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Farbe hinzufügen', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Farbe hinzufügen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', '3D-Beleuchtung', 'ABAP'], ['ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltung', 'ein Skript anpassen', 'Buchhaltung', 'Anpassung an Veränderungen im Marketing', 'ein Skript anpassen', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken'], ['ein Skript anpassen', 'ein Skript anpassen', 'ein Skript anpassen', 'Requisiten anpassen', 'Requisiten anpassen', '3D-Beleuchtung', 'Akupressur', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Fitnessübungen anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'ABAP', '3D-Beleuchtung', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Akupressur', 'ABAP', 'ABAP', 'ABAP', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Farbe hinzufügen', 'Requisiten anpassen', 'Buchhaltung', 'Farbe hinzufügen', 'Requisiten anpassen', 'Buchhaltung', 'ABAP', 'ABAP', '3D-Beleuchtung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing'], ['ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltung', 'Buchhaltungstechniken', 'Buchhaltung', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltung', 'Buchhaltungstechniken'], ['Akupressur', 'Akupressur', 'Systemkomponente erwerben', 'ABAP', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Hörtests anpassen', 'Systemkomponente erwerben', 'ABAP', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Requisiten anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Buchhaltung', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken'], ['Akupressur', 'Buchhaltung', 'Akupressur', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Farbe hinzufügen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben'], ['Buchhaltung', 'Buchhaltung', 'Akupressur', 'Buchhaltungstechniken', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Hörtests anpassen', 'ABAP', 'ABAP', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Hörtests anpassen', 'Akupressur', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Akupressur', 'ABAP', 'Akupressur', 'ABAP', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken', 'Akupressur', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Schauspieltechniken', 'Schauspieltechniken', 'ABAP', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Schauspieltechniken', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben'], ['Farbe hinzufügen', 'Absorb (Lernmanagementsysteme)', 'Farbe hinzufügen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben'], ['Prozesse der Buchhaltungsabteilung', 'Buchhaltung', 'Buchhaltungstechniken', 'ABAP', 'Buchhaltungstechniken', 'Akupressur', 'ABAP', 'Akupressur', 'ABAP', 'Akupressur', 'Akupressur', 'Buchhaltungstechniken', 'ABAP', 'ABAP', 'Buchhaltung', 'Requisiten anpassen', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Farbe hinzufügen', 'Anpassung an Veränderungen im Marketing', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Texturierung', 'Prozesse der Buchhaltungsabteilung', 'Buchhaltung', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Akupressur', 'Akupressur', 'ABAP', 'Anpassung an Veränderungen im Marketing', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Computerkomponenten hinzufügen', 'ABAP', 'ein Skript anpassen', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Sätze anpassen', 'Akupressur', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung', 'Buchhaltung', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing', 'Systemkomponente erwerben', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'ein Skript anpassen', 'zuverlässig handeln', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', '3D-Druckverfahren', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Akutversorgung', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Buchhaltungstechniken', 'Sätze anpassen', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', '3D-Beleuchtung', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Akupressur', 'Akupressur', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'zuverlässig handeln', 'ein Skript anpassen', 'ein Skript anpassen', 'Akupressur', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken'], ['Anpassung an Veränderungen im Marketing', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Veränderungen im Marketing', 'Systemkomponente erwerben', 'ein Skript anpassen', 'ein Skript anpassen'], ['ein Skript anpassen', 'ABAP', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ABAP', '3D-Druckverfahren', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Schauspiel- und Regietechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Schauspieltechniken', 'ein Skript anpassen', 'ein Skript anpassen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Farbe hinzufügen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', '3D-Beleuchtung', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben'], ['Buchhaltung', 'ABAP', 'Buchhaltung', 'Anpassung an Veränderungen im Marketing', 'Schauspieltechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Buchhaltungstechniken', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akustiktechnik', '3D-Modellierung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Akupressur', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)'], ['Buchhaltung', 'Buchhaltung', 'ein Skript anpassen', 'Akupressur', 'Buchhaltung', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Farbe hinzufügen', 'Buchhaltungstechniken', 'Akupressur', 'ABAP', 'Akupressur', 'Buchhaltung', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Prozesse der Buchhaltungsabteilung', 'ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Veränderungen auf einem Boot', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Veränderungen im Marketing', 'ABAP', 'Hörtests anpassen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Sätze anpassen', 'Anpassung an Veränderungen in der Forstwirtschaft', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ABAP', 'Akupressur', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['ein Skript anpassen', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', 'ein Skript anpassen', '3D-Beleuchtung', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Buchhaltungstechniken', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Akupressur', 'Akupressur', 'ein Skript anpassen', 'Buchhaltung', 'ein Skript anpassen', 'Schauspieltechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'ein Skript anpassen', 'ein Skript anpassen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'ABAP', 'Akupressur', 'Anpassung an Veränderungen im Marketing', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Buchhaltungstechniken'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', '3D-Modellierung', 'Anpassung an Veränderungen im Marketing', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'ABAP', 'ABAP', 'ABAP', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)'], ['ein Skript anpassen', 'Buchhaltung', 'ein Skript anpassen', 'ABAP', 'Buchhaltungstechniken', 'Schauspieltechniken', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Buchhaltung', 'Buchhaltung', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Buchhaltungstechniken'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ABAP', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'ABAP', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Akupressur', 'Buchhaltung', 'Buchhaltung', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Hinzufügen von Spulen zu Keramikarbeiten', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Akupressur', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ABAP', 'ABAP', 'ABAP', 'Sätze anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Farbe hinzufügen', 'Buchhaltung', 'Buchhaltung', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'ABAP', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'ABAP', 'Akupressur', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['ein Skript anpassen', 'Präzisionswaffen', 'Buchhaltungstechniken', 'Prozesse der Buchhaltungsabteilung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Schauspieltechniken', 'Sätze anpassen', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', '3D-Beleuchtung', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Hörtests anpassen', 'Akupressur', '3D-Texturierung', 'Hörtests anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Buchhaltung', 'ABAP', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Buchhaltungstechniken'], ['Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ABAP', 'ABAP', 'Buchhaltung', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen', 'ein Skript anpassen', 'Sätze anpassen', 'ein Skript anpassen', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Prozesse der Buchhaltungsabteilung', 'Schauspieltechniken', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Farbe hinzufügen', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Prozesse der Buchhaltungsabteilung'], ['Systemkomponente erwerben', 'Hörtests anpassen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Hörtests anpassen', 'Akupressur', 'Buchhaltungstechniken', 'Akupressur', 'Buchhaltung', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Beleuchtung', 'Akupressur', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', '3D-Beleuchtung', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltung', 'Akupressur', 'ABAP', 'Farbe hinzufügen', 'Akupressur', 'Sätze anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ein Skript anpassen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Akupressur', 'Akupressur', 'Buchhaltung', 'Buchhaltung', '3D-Modellierung', 'Akutversorgung', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltungstechniken', 'ABAP', 'Buchhaltungstechniken', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Buchhaltungstechniken'], ['ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Buchhaltung', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltung'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'ABAP', 'Akupressur', 'Buchhaltungstechniken', 'Akupressur', 'Systemkomponente erwerben', 'Schauspieltechniken', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Buchhaltung', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'ein Skript anpassen'], ['Buchhaltung', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Buchhaltung'], ['Absorb (Lernmanagementsysteme)', 'ABAP', 'ein Skript anpassen', 'ABAP', 'Text kulturell anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Prozesse der Buchhaltungsabteilung', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Systemkomponente erwerben'], ['Buchhaltung', 'Systemkomponente erwerben', 'Buchhaltung', 'Anpassung an Veränderungen im Marketing', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', '3D-Modellierung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'ABAP', 'ABAP', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Veränderungen im Marketing', 'Buchhaltungstechniken'], ['ein Skript anpassen', 'ein Skript anpassen', 'Präzisionswaffen', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'ein Skript anpassen', 'ABAP', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'ein Skript anpassen', 'Systemkomponente erwerben', 'Akupressur', 'Akupressur', 'Buchhaltung', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Hörtests anpassen', 'Akupressur', 'Buchhaltung'], ['Buchhaltungstechniken', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'ABAP', 'Anpassung an Veränderungen im Marketing', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Computerkomponenten hinzufügen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben'], ['ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Akupressur', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken'], ['Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABBYY FineReader', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Akupressur', 'Akupressur', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)'], ['Buchhaltungstechniken', 'ein Skript anpassen', 'ein Skript anpassen', 'ABAP', 'ein Skript anpassen', 'Akupressur', 'Akupressur', 'Akupressur', 'Akupressur', 'Buchhaltung', 'Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', '3D-Beleuchtung', 'Farbe hinzufügen', 'ein Skript anpassen', 'ein Skript anpassen', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Systemkomponente erwerben', 'ein Skript anpassen', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Buchhaltung'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung', 'Systemkomponente erwerben', 'ABAP', 'Systemkomponente erwerben', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Buchhaltung', 'ABAP', 'Buchhaltung', 'ABAP', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Absorb (Lernmanagementsysteme)', 'Buchhaltung', 'Akupressur', 'ein Skript anpassen', 'Akupressur', 'ABAP', 'Buchhaltungstechniken', 'Akupressur', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'ABAP', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Veränderungen in der Forstwirtschaft', 'Buchhaltungstechniken', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Absorb (Lernmanagementsysteme)', 'Buchhaltungstechniken', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Buchhaltungstechniken'], ['Absorb (Lernmanagementsysteme)', 'Absorb (Lernmanagementsysteme)', 'Akupressur', 'Sätze anpassen', 'Buchhaltungstechniken', 'Farbe hinzufügen', 'ein Skript anpassen', 'ein Skript anpassen', 'Systemkomponente erwerben'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'ein Skript anpassen', 'Akupressur', 'ein Skript anpassen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung'], ['Systemkomponente erwerben', 'Systemkomponente erwerben', 'Akupressur', 'ein Skript anpassen', 'Absorb (Lernmanagementsysteme)', 'Anpassung an Änderungen in technologischen Entwicklungsplänen', 'Systemkomponente erwerben', 'Buchhaltungstechniken', 'Systemkomponente erwerben', 'Systemkomponente erwerben', 'Buchhaltung'], ['ein Skript anpassen', 'ein Skript anpassen', 'Buchhaltung', 'Buchhaltungstechniken', 'Buchhaltung', 'Buchhaltungstechniken']]

In [2]:
len(extracted_spans)

95

In [3]:
from pathlib import Path
files = Path("C:/Users/karam/PycharmProjects/skillExtraction/itstellen").glob("*.txt")
files2 = Path("C:/Users/karam/PycharmProjects/skillExtraction/karriere/out").glob("*.txt")
numberOfJobpostings = list(files)+list(files2)
len(numberOfJobpostings)

2838